In [6]:
# 모델 구축
# 입력값 토큰화, 임베딩
# 훈련을 정의하고 학습
# 평가
# 예측하기 위한 데이터를 토큰화, 임베딩후 predict
# 라벨 출력

In [7]:
from tensorflow.keras import Model, layers

# 입력 단어는 4글자 영단어, 전체문자수는 172개(SEP 포함)
enc_input = layers.Input(shape=(4, 172))
_, state_h, state_c = layers.LSTM(128, return_state=True)(enc_input)

dec_input = layers.Input(shape=(3, 172)) # 한글 단어 2자와 start 토큰 1개
lstm_out = layers.LSTM(128, return_sequences=True)(dec_input, initial_state=[state_h, state_c])

dec_output = layers.Dense(172, activation='softmax')(lstm_out)

model = Model(inputs=[enc_input, dec_input], outputs=dec_output)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 4, 172)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 3, 172)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 128),     │    154,112 │ input_layer_2[0]… │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 3, 128)    │    154,112 │ input_layer_3[0]… │
│                     │                   │            │ lstm_2[0][1],     │
│                     │                   │            │ lstm_2[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 3, 172)    │     22,188 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 330,412 (1.26 MB)

 Trainable params: 330,412 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam') # metrics가 없으면 훈련시 loss만 출력됨

In [9]:
# 문자배열 생성
import pandas as pd
import numpy as np
# <START>, <END>, <PAD>
arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
arr2 = pd.read_csv('korean.csv', header=None)
# print(arr2[0].values.tolist())
num_to_char = arr1 + arr2[0].values.tolist()
print(len(num_to_char))
char_to_num = {char:i for i, char in enumerate(num_to_char)}
# print(char_to_num)

171


In [10]:
# 학습용 단어셋 불러오기
raw = pd.read_csv('translate.csv', header=None)
eng_kor = raw.values.tolist()
print(len(eng_kor)) # 학습할 전체 단어 수 110개

110


In [11]:
# 단어를 숫자 배열로 변환
temp_eng = 'love'
temp_eng_n = [char_to_num[c] for c in temp_eng]
print(temp_eng_n)
np.eye(172)[temp_eng_n].shape

[14, 17, 24, 7]


(4, 172)

In [12]:
# 단어를 원-핫 인코딩된 배열로 변환
def encode(eng_kor):
    enc_in = []
    dec_in = []
    rnn_out = [] # decoder output
    for seq in eng_kor:
        eng = [char_to_num[c] for c in seq[0]]
        enc_in.append(np.eye(172)[eng])

        kor = [char_to_num[c] for c in ('S'+seq[1])]
        dec_in.append(np.eye(172)[kor])

        target = [char_to_num[c] for c in (seq[1] + 'E')]
        rnn_out.append(target)

    enc_in = np.array(enc_in)
    dec_in = np.array(dec_in)
    rnn_out = np.array(rnn_out)
    rnn_out = np.expand_dims(rnn_out, axis=2)
    return enc_in, dec_in, rnn_out

In [13]:
sample = [['word', '단어']]
encode(sample)[2]

array([[[ 61],
        [114],
        [  1]]])

In [14]:
X_enc, X_dec, y_rnn = encode(eng_kor)

In [15]:
%%time
model.fit([X_enc, X_dec], y_rnn, epochs=500)

Epoch 1/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 5.1405
Epoch 2/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 5.1032
Epoch 3/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 5.0564
Epoch 4/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4.9781
Epoch 5/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 4.8243
Epoch 6/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 4.4902
Epoch 7/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.8343
Epoch 8/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.5412
Epoch 9/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.6247
Epoch 10/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.4396
Epoch 11/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.3910
Epoch 12/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 3.3973
Epoch 13/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.3485
Epoch 14/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.3140
Epoch 15/500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 3.3043
Epoch 16/500
4/4 ━━

In [16]:
enc_in, dec_in, _ = encode([['life', 'PP']])
# print(enc_in)
pred = model.predict([enc_in, dec_in])
print(pred.shape)
word = np.argmax(pred[0], axis=-1)
# print(pred[0][0])
print(np.argmax(pred[0][0]))
print(word)
num_to_char[word[0]], num_to_char[word[1]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
(1, 3, 172)
99
[99 80  1]


('생', '명')

In [17]:
from numpy.random import randint
pick = randint(0, len(eng_kor), 5)
choose = [ [eng_kor[i][0], 'PP'] for i in pick]
print(choose)

[['cook', 'PP'], ['exit', 'PP'], ['menu', 'PP'], ['tool', 'PP'], ['farm', 'PP']]


In [18]:
enc_in, dec_in, _ = encode(choose)
pred = model.predict([enc_in, dec_in])
print(pred.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
(5, 3, 172)


In [19]:
for i in range(len(choose)):
    eng = choose[i][0]
    word = np.argmax(pred[i], axis=-1)
    kor = ''
    for j in range(2):
        kor = kor + num_to_char[word[j]]
    print(eng, kor)

cook 요리
exit 탈출
menu 메뉴
tool 도구
farm 농장
